In [1]:
! pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [4]:
encoded = tokenizer.encode("Do not meddle in the affairs of wizards")

In [5]:
print(tokenizer.convert_ids_to_tokens(encoded))


['[CLS]', 'Do', 'not', 'me', '##ddle', 'in', 'the', 'affairs', 'of', 'wizard', '##s', '[SEP]']


In [6]:
! pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=b7211151af5f7f3b9bbaedc705478f5d4dd842657a0fe218a0bee525e0f67af0
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the pre-trained tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Input sentence
sentence = "Time after time, time flies like an arrow, but fruit flies like a banana."

# Words of interest
words_of_interest = ["time", "flies", "like"]

# Tokenize the input sentence
tokens = tokenizer.tokenize(sentence)

# Initialize a dictionary to store the embeddings for each word
word_embeddings = {word: [] for word in words_of_interest}

# Disable gradient calculation for inference
with torch.no_grad():
    # Tokenize the input text and convert to a tensor
    encoded_input = tokenizer(sentence, return_tensors='pt')

    # Get the model's output (hidden states)
    outputs = model(**encoded_input)
    last_hidden_states = outputs.last_hidden_state

    # Extract the embeddings for the words of interest
    for i, token in enumerate(tokens):
        if token.lower() in words_of_interest:
            word_embeddings[token.lower()].append(last_hidden_states[0, i])

# Compute cosine similarities between the word embeddings
for word, embeddings in word_embeddings.items():
    for i in range(len(embeddings)):
        for j in range(i + 1, len(embeddings)):
            similarity = cosine_similarity(embeddings[i].reshape(1, -1), embeddings[j].reshape(1, -1))
            print(f"Cosine similarity between '{word}' in position {i + 1} and position {j + 1}: {similarity[0][0]}")


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Cosine similarity between 'time' in position 1 and position 2: 0.21499860286712646
Cosine similarity between 'time' in position 1 and position 3: 0.3339453637599945
Cosine similarity between 'time' in position 2 and position 3: 0.4939243495464325
Cosine similarity between 'flies' in position 1 and position 2: 0.5807819962501526
Cosine similarity between 'like' in position 1 and position 2: 0.9300364255905151


In [9]:
from transformers import BertTokenizer, BertModel
import torch

# Initialize the tokenizer and model from the BERT family
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Sentences to analyze
sentence1 = "Time flies like an arrow"
sentence2 = "Fruit flies like a banana"

# Tokenize and encode sentences for BERT input
tokens1 = tokenizer.encode_plus(sentence1, return_tensors='pt')
tokens2 = tokenizer.encode_plus(sentence2, return_tensors='pt')

# Get the embeddings from the BERT model
outputs1 = model(**tokens1)
outputs2 = model(**tokens2)

# Retrieve the embeddings for the word "like" for each sentence
# Assuming 'like' is not the first word and does not get split into subwords.
like_index1 = tokens1['input_ids'][0].tolist().index(tokenizer.encode('like', add_special_tokens=False)[0])
like_index2 = tokens2['input_ids'][0].tolist().index(tokenizer.encode('like', add_special_tokens=False)[0])

like_embedding1 = outputs1.last_hidden_state[0, like_index1]
like_embedding2 = outputs2.last_hidden_state[0, like_index2]

# Compare the embeddings, e.g., by using cosine similarity
cosine_similarity = torch.nn.CosineSimilarity(dim=0)
similarity = cosine_similarity(like_embedding1, like_embedding2).item()

print(f"Cosine similarity between 'like' in both contexts: {similarity}")

Cosine similarity between 'like' in both contexts: 0.7066124081611633


In [7]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.37173552e-02 -4.28515449e-02 -1.56286024e-02  1.40537303e-02
  3.95537727e-02  1.21796280e-01  2.94334106e-02 -3.17524187e-02
  3.54959629e-02 -7.93139935e-02  1.75878741e-02 -4.04369719e-02
  4.97259349e-02  2.54912246e-02 -7.18700588e-02  8.14968869e-02
  1.47069141e-03  4.79626991e-02 -4.50336412e-02 -9.92174670e-02
 -2.81769745e-02  6.45046085e-02  4.44670543e-02 -4.76217009e-02
 -3.52952331e-02  4.38671783e-02 -5.28566055e-02  4.33063833e-04
  1.01921506e-01  1.64072234e-02  3.26996595e-02 -3.45986746e-02
  1.21339476e-02  7.94870779e-02  4.58345609e-03  1.57777797e-02
 -9.68206208e-03  2.87625659e-02 -5.05805984e-02 -1.55793717e-02
 -2.87906546e-02 -9.62280575e-03  3.15556750e-02  2.27349028e-02
  8.71449187e-02 -3.85027491e-02 -8.84718448e-02 -8.75498448e-03
 -2.12343335e-02  2.08923239e-02 -9.02077407e-02 -5.25732562e-02
 -1.05638904e-02  2.88310610e-02 -1.61455162e-02  6.17837207e-03
 -1.23234